In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pyart
import matplotlib.pyplot as plt
import os
from PIL import Image
import glob
import numpy as np
import scipy.io


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
def plot_data(infilename, outpng):
  radar = pyart.io.read_nexrad_archive(infilename)
  display = pyart.graph.RadarMapDisplay(radar)
  fig = plt.figure(figsize=(10, 10))
  display.plot_ppi_map('reflectivity')
  fig.savefig(outpng)
  plt.close(fig)

In [4]:
plot_data('Del_rio_data/KDFX20220701_000041_V06','fig1')

In [5]:
del_files=os.listdir("Del_rio_data")
new_braunfels=os.listdir("New_Braunfels")

In [ ]:
images1 = "del_rio_radar_plots"
i = 0
for volumefile in del_files[:118]:
    try: 
        plot_data(os.path.join("Del_rio_data",volumefile), os.path.join(images1,"fig"+str(i)))
    except:
        continue
    i+=1

In [ ]:
print("Radar images for Del_rio generated")

In [9]:
images1 = "new_braunfels_radar_plots"
i = 0
for volumefile in new_braunfels[:118]:
    try: 
        plot_data(os.path.join("New_Braunfels",volumefile), os.path.join(images1,"fig"+str(i)))
    except:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        continue
    i+=1

In [ ]:
print("Radar images for new_braunfels generated")

In [6]:
def png_gif(png_path, gif_name):
    frames = []
    imgs = glob.glob(png_path)
    for i in imgs:
        new_frame = Image.open(i)
        frames.append(new_frame)

    frames[0].save(gif_name, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)

In [10]:
png_gif("del_rio_radar_plots/*.png", "del_rio_gif.gif")
png_gif("new_braunfels_radar_plots/*.png", "new_braunfels_gif.gif")

In [8]:
def to_cartesian_array(radar, size):
    '''
    Returns:
        numpy array with the reflectivity data in cartesian coordinates
        '''
    grids = pyart.map.grid_from_radars(                             #  convert to cartesian
        radar, 
        grid_shape = (1, size[0], size[1]),                          # define shape of frames (first dim is Z)
        grid_limits = ((radar.altitude['data'][0], 20000.0),         # z
                       (-150000, 150000),                            # y
                       (-150000, 150000)),                           # x
        fields=['reflectivity'],                                     # which data
        gridding_algo= 'map_gates_to_grid',                          # gridding algorithm
        weighting_function='BARNES2')                                # interpolation fx
    img_mtx = grids.fields['reflectivity']['data'][0,:,:]            # grab the reflectivity data (2D image)
    img_mtx = np.clip(img_mtx,0,70)                                  # truncate the dbz to lower and upper bounds to produce same colored images when normalizing 
    img_mtx = np.ma.filled(img_mtx, fill_value=0)                    # complate missing values with 0dBZ
    img_mtx = np.rint(img_mtx/70*255)                                # re-scale to 0-255
    img_mtx = img_mtx.astype('uint8')                                # use uint8 to be efficient
    return img_mtx

def to_numpy_array(path, size =(64,64)):
    files_c = len(os.listdir(path))
    data   = np.zeros((1,files_c,size[0],size[1]))
    titles = np.empty(files_c, dtype=object)
    i = 0
    for file in os.listdir(path):
        try:
            radar = pyart.io.read_nexrad_archive(os.path.join(path, file)) # open the ar2v file
        except:
            continue
        data[0,i] = to_cartesian_array(radar, size)
        titles[i] = file
        i+=1
    np.save(os.path.join("array_input", path), data)
    return 


In [9]:
to_numpy_array("Del_rio_data")
to_numpy_array("New_Braunfels")

In [13]:
del_rio_history = np.load(os.path.join("array_input", "Del_rio_data.npy"))
New_Braunfels_history = np.load(os.path.join("array_input", "New_Braunfels.npy"))

In [17]:
print(del_rio_history.shape)
print(New_Braunfels_history.shape)

(1, 186, 64, 64)
(1, 138, 64, 64)


In [21]:
New_Braunfels_history[0][0]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 26.,  0.,  0.],
       ...,
       [41., 46., 54., ...,  0.,  0.,  0.],
       [61., 68., 76., ...,  0.,  0.,  0.],
       [75., 87., 93., ...,  0.,  0.,  0.]])